In [1]:
import  pandas  as  pd

In [2]:
df_code_postal = pd.read_csv('../Data/adresses-69.csv',  sep=';')
df_code_postal.head()


C:\Users\ediad\AppData\Local\Temp\ipykernel_3748\3924582151.py:1: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_code_postal = pd.read_csv('../Data/adresses-69.csv',  sep=';')


,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles
0,69002_x415an_00003,NaN,3,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,46.277603,entrée,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN
1,69002_x415an_00111,NaN,111,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,46.278509,entrée,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN
2,69002_x415an_00165,NaN,165,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,46.278966,entrée,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN
3,69002_x415an_01148,NaN,1148,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,46.287092,entrée,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN
4,69002_wit9km_00007,NaN,7,NaN,Chemin des Dalhias,69790,69002,Aigueperse,NaN,NaN,...,46.276468,entrée,NaN,NaN,AIGUEPERSE,CHEMIN DES DALHIAS,commune,commune,1,NaN


In [3]:
code_postals = sorted(df_code_postal['code_postal'].unique().tolist())

In [4]:
df  = pd.read_csv('../Data/dpe-v2-logements-existants.csv')

res = list(df.columns.values)
res = [str(i) for i in res if i != '_score']
listeColumn =  ",".join(res[:100])
listeColumn

C:\Users\ediad\AppData\Local\Temp\ipykernel_3748\752924344.py:1: DtypeWarning: Columns (12,48,190,191,192,193) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv('../Data/dpe-v2-logements-existants.csv')


"N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),Méthode_application_DPE,N°_immatriculation_copropriété,Invariant_fiscal_logement,N°_RPLS_logement,Etiquette_GES,Etiquette_DPE,Année_construction,Type_bâtiment,Type_installation_chauffage,Type_installation_ECS_(général),Période_construction,Hauteur_sous-plafond,Nombre_appartement,Nombre_niveau_immeuble,Nombre_niveau_logement,Surface_habitable_immeuble,Surface_habitable_logement,Surface_tertiaire_immeuble,Classe_inertie_bâtiment,Typologie_logement,Position_logement_dans_immeuble,Classe_altitude,Zone_climatique_,Adresse_brute,Nom__commune_(BAN),Code_INSEE_(BAN),N°_voie_(BAN),Identifiant__BAN,Adresse_(BAN),Code_postal_(BAN),Score_BAN,Nom__rue_(BAN),Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),Code_postal_(brut),N°_étage_appartement,Nom_résidence,Complément_d'adresse_bâtiment,Cage_

In [ ]:
import requests
import pandas as pd
import os
import concurrent.futures


if os.path.exists('../Data/data69.csv'):
    os.remove('../Data/data69.csv')

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines"

# Liste pour stocker les résultats
all_results = []
somme = 0
compteur =  0

df_final  =  pd.DataFrame(columns=res)

 

def fetch_data(code_postal):
    global  somme
    global  df_final
    global  compteur
    print(code_postal)
    params = {
        "page": 1,
        "size": 1000,
        "q": str(code_postal),
        "select": listeColumn,
        "q_fields": "Code_postal_(BAN)",
    }

    # Effectuer la requête GET avec les paramètres
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        print(f"Nombre total de lignes pour le code postal {code_postal} : {response.json()['total']}")
        somme += response.json()['total']
        temp = pd.DataFrame(response.json()['results'])
        compteur = compteur + len(temp)
        
        df_final = pd.concat([df_final , temp])
        print('\nNombre de  lignes : ', len(df_final))
        # Pagination
        next_url = response.json().get('next')
        while next_url:
            response = requests.get(next_url)
            if response.status_code == 200:
                temp = pd.DataFrame(response.json()['results'])
                 
                df_final = pd.concat([df_final , temp])
                print('\nNombre de  lignes : ', len(df_final))
                
                next_url = response.json().get('next')
            else:
                print(f"Erreur lors de la requête pour le code postal {code_postal}  ")
                print(response.text)
                break
    else:
        print(f"Erreur pour le code postal {code_postal} : {response.status_code}")

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(fetch_data, code_postal) for code_postal in code_postals]
    for future in concurrent.futures.as_completed(futures):
        future.result()


df_final.to_csv('../Data/data69-existants.csv', mode='a', index=False, header=True, encoding='utf-8-sig', sep=';')

print('Total normal: ', somme)
print('Total CSV: ', len(df_final))
del df_final

69001
69002
69003
69004
69005
69006
69007
69008
69009
69100
69110
69115
69120
69124
69126
69130
69140
69150
69160
69170
69190
69200
69210
69220
69230
69240
69250
69260
69270
69280
69290
69300
69310
69320
69330
69340
69350
69360
69370
69380
69390
69400
69410
69420
69430
69440
69450
69460
69470
69480
69490
69500
69510
69520
69530
69540
69550
69560
69570
69580
69590
69600
69610
69620
69630
69640
69650
69660
69670
69680
69690
69700
69720
69730
69740
69760
69770
69780
69790
69800
69820
69830
69840
69850
69860
69870
69890
69910
69930
69960
69970
Nombre total de lignes pour le code postal 69124 : 181

Nombre de  lignes :  181


C:\Users\ediad\AppData\Local\Temp\ipykernel_3748\2341529338.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final , temp])


Nombre total de lignes pour le code postal 69280 : 330
Nombre total de lignes pour le code postal 69115 : 27

Nombre de  lignes :  208

Nombre de  lignes :  538
Nombre total de lignes pour le code postal 69820 : 117
Nombre total de lignes pour le code postal 69770 : 165

Nombre de  lignes :  655
Nombre total de lignes pour le code postal 69930 : 165

Nombre de  lignes :  820

Nombre de  lignes :  820
Nombre total de lignes pour le code postal 69670 : 281

Nombre de  lignes :  1101
Nombre total de lignes pour le code postal 69660 : 400
Nombre total de lignes pour le code postal 69860 : 105

Nombre de  lignes : Nombre total de lignes pour le code postal 69470 : 455
 1206

Nombre de  lignes :  1501
Nombre total de lignes pour le code postal 69840 : 201

Nombre de  lignes :  1661
Nombre total de lignes pour le code postal 69690 : 328
Nombre total de lignes pour le code postal 69910 : 115
Nombre total de lignes pour le code postal 69580 : 507


C:\Users\ediad\AppData\Local\Temp\ipykernel_3748\2341529338.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final , temp])


Nombre total de lignes pour le code postal 69790 : 112
Nombre total de lignes pour le code postal 69760 : 363

Nombre de  lignes :  1862
Nombre total de lignes pour le code postal 69610 : 440
Nombre total de lignes pour le code postal 69890 : 334

Nombre de  lignes :  1776
Nombre total de lignes pour le code postal 69460 : 615

Nombre de  lignes :  1989


C:\Users\ediad\AppData\Local\Temp\ipykernel_3748\2341529338.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final , temp])


Nombre total de lignes pour le code postal 69850 : 278

Nombre de  lignes :  2168

Nombre de  lignes :  2139
Nombre total de lignes pour le code postal 69370 : 481
Nombre total de lignes pour le code postal 69640 : 386

Nombre de  lignes :  1974

Nombre de  lignes :  2216

Nombre de  lignes :  2110

Nombre de  lignes :  2446


C:\Users\ediad\AppData\Local\Temp\ipykernel_3748\2341529338.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final , temp])



Nombre de  lignes :  2602
Nombre total de lignes pour le code postal 69630 : 547
Nombre total de lignes pour le code postal 69450 : 429

Nombre de  lignes :  2783

Nombre de  lignes :  2697

Nombre de  lignes :  3126

Nombre de  lignes :  3244
Nombre total de lignes pour le code postal 69126 : 374

Nombre de  lignes :  3618
Nombre total de lignes pour le code postal 69390 : 1312

Nombre de  lignes :  4618
Nombre total de lignes pour le code postal 69540 : 712
Nombre total de lignes pour le code postal 69970 : 395

Nombre de  lignes :  5330
Nombre total de lignes pour le code postal 69560 : 406

Nombre de  lignes :  5013
Nombre total de lignes pour le code postal 69550 : 546
Nombre total de lignes pour le code postal 69650 : 595
Nombre total de lignes pour le code postal 69590 : 540
Nombre total de lignes pour le code postal 69430 : 611
Nombre total de lignes pour le code postal 69005 : 14942

Nombre de  lignes :  5736
Nombre total de lignes pour le code postal 69420 : 669
Nombre total

In [9]:
# import requests
# import pandas as pd
# import os

# # URL de base de l'API
# base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines"

# # Liste pour stocker les résultats
# all_results = []
# somme = 0

 
# for code_postal in code_postals:
#         # print(code_postal)
#         params = {
#             "page": 1,
#             "size": 1000,
#             "select": listeColumn,
#             "q": str(code_postal),
#             "q_fields": "Code_postal_(BAN)"
            
#             }

#         # Effectuer la requête GET avec les paramètres
#         response = requests.get(base_url, params=params)
#         print(code_postal)
#         if(response.status_code == 200):
#             print(f"Nombre total de lignes pour le code postal {code_postal} : {response.json()['total']}")
#             somme += response.json()['total']
#             print('Total : ', somme)
#             temp = pd.DataFrame(response.json()['results'])

#             if not os.path.exists('../Data/data69.csv'):
#                 temp.to_csv('../Data/data69.csv', sep=';', mode='a',header=True,
#                             encoding='utf-8-sig', index=False)
#             else:
#                 temp.to_csv('../Data/data69.csv', sep=';', mode='a',header=False,
#                             encoding='utf-8-sig', index=False)
#         try:
#             next = True  if  response.json()['next'] else False
#         except:
#             next = False
            
#         while next:
#             next_res =  requests.get(response.json()['next'])
            
#             print(f"\tNombre total de lignes pour le code postal {code_postal} restant: {next_res.json()['total']}")
#             if(next_res.status_code == 200):
#                 response = next_res
#                 temp = pd.DataFrame(response.json()['results'])
#                 temp.to_csv('../Data/data69.csv', sep=';', mode='a',header=False,
#                             encoding='utf-8-sig', index=False)
                

#                 try:
#                    next = True  if  response.json()['next'] else False
#                 except:
#                     next = False    

#             else:
#                 print(f"Erreur lors de la requête pour le code postal {code_postal}")
#                 print(response.text)
#                 break


# print('Total : ', somme)

In [20]:
# import requests
# import pandas as pd
# import os
# import concurrent.futures


# if os.path.exists('../Data/data69.csv'):
#     os.remove('../Data/data69.csv')

# # URL de base de l'API
# base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines"

# # Liste pour stocker les résultats
# all_results = []
# somme = 0

# df_final  =  pd.DataFrame(columns=res)
# # def  cre
 

# def fetch_data(code_postal):
#     global  somme
#     global  df_final
#     print(code_postal)
#     params = {
#         "page": 1,
#         "size": 10,
#         "q": str(code_postal),
#         "select": listeColumn,
#         "q_fields": "Code_postal_(BAN)",
#     }

#     # Effectuer la requête GET avec les paramètres
#     response = requests.get(base_url, params=params)

#     if response.status_code == 200:
#         print(f"Nombre total de lignes pour le code postal {code_postal} : {response.json()['total']}")
#         somme += response.json()['total']
#         # print('Total : ', somme)

#         temp = pd.DataFrame(response.json()['results'])
#         df_final = pd.concat([df_final , temp])

        
#         # if not os.path.exists('../Data/data69.csv'):
#         #     # temp.fillna(inplace=True)
#         #     temp.to_csv('../Data/data69.csv', mode='a', index=False, header=True, encoding='utf-8-sig', sep=';')
#         #     print('done')

#         # else:   
#         #     # temp.fillna(inplace=True)
#         #     temp.to_csv('../Data/data69.csv', mode='a', header=False, index=False, encoding='utf-8-sig', sep=';')
#         #     print('done')


#         # Pagination
#         next_url = response.json().get('next')
#         while next_url:
#             response = requests.get(next_url)
#             if response.status_code == 200:
#                 temp = pd.DataFrame(response.json()['results'])
#                 df_final = pd.concat([df_final , temp])
                
#                 # temp.to_csv('../Data/data69.csv', mode='a', header=False, index=False, encoding='utf-8-sig', sep=';')
#                 next_url = response.json().get('next')
#                 print('done')
#             else:
#                 print(f"Erreur lors de la requête pour le code postal {code_postal}  ")
#                 print(response.text)
#                 break
#     else:
#         print(f"Erreur pour le code postal {code_postal} : {response.status_code}")

# with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
#     futures = [executor.submit(fetch_data, code_postal) for code_postal in code_postals]
#     for future in concurrent.futures.as_completed(futures):
#         future.result()


# df_final.to_csv('../Data/data69.csv', mode='a', index=False, header=True, encoding='utf-8-sig', sep=';')
# del df_final
# print('Total : ', somme)

69001
69002
69003
69004
69005
69006
69007
69008
69009
69100
69110
69115
69120
69124
69126
69130
69140
69150
69160
69170
69190
69200
69210
69220
69230
69240
69250
69260
69270
69280
69290
69300
69310
69320
69330
69340
69350
69360
69370
69380
69390
69400
69410
69420
69430
69440
69450
69460
69470
69480
69490
69500
69510
69520
69530
69540
69550
69560
69570
69580
69590
69600
69610
69620
69630
69640
69650
69660
69670
69680
69690
69700
69720
69730
69740
69760
69770
69780
69790
69800
69820
69830
69840
69850
69860
69870
69890
69910
69930
69960
69970
Nombre total de lignes pour le code postal 69115 : 27
Nombre total de lignes pour le code postal 69190 : 3185
Nombre total de lignes pour le code postal 69290 : 2788
Nombre total de lignes pour le code postal 69380 : 1855
Nombre total de lignes pour le code postal 69006 : 9304
Nombre total de lignes pour le code postal 69620 : 752
Nombre total de lignes pour le code postal 69690 : 328


C:\Users\ediad\AppData\Local\Temp\ipykernel_31424\394900118.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final , temp])


Nombre total de lignes pour le code postal 69124 : 181
Nombre total de lignes pour le code postal 69280 : 330
Nombre total de lignes pour le code postal 69930 : 165
Nombre total de lignes pour le code postal 69460 : 615
Nombre total de lignes pour le code postal 69630 : 547
Nombre total de lignes pour le code postal 69820 : 117
Nombre total de lignes pour le code postal 69490 : 540
Nombre total de lignes pour le code postal 69230 : 1917
Nombre total de lignes pour le code postal 69260 : 439
Nombre total de lignes pour le code postal 69440 : 998
Nombre total de lignes pour le code postal 69700 : 2575
Nombre total de lignes pour le code postal 69740 : 847
Nombre total de lignes pour le code postal 69002 : 8503
Nombre total de lignes pour le code postal 69450 : 429
Nombre total de lignes pour le code postal 69008 : 13703
Nombre total de lignes pour le code postal 69360 : 1490
Nombre total de lignes pour le code postal 69890 : 334
Nombre total de lignes pour le code postal 69850 : 278
Nomb

C:\Users\ediad\AppData\Local\Temp\ipykernel_31424\394900118.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final , temp])


Nombre total de lignes pour le code postal 69340 : 1383
Nombre total de lignes pour le code postal 69007 : 21149
Nombre total de lignes pour le code postal 69840 : 201
Nombre total de lignes pour le code postal 69130 : 2343
Nombre total de lignes pour le code postal 69390 : 1312
Nombre total de lignes pour le code postal 69300 : 5997
Nombre total de lignes pour le code postal 69003 : 18890
Nombre total de lignes pour le code postal 69870 : 492
Nombre total de lignes pour le code postal 69520 : 1038
Nombre total de lignes pour le code postal 69610 : 440
Nombre total de lignes pour le code postal 69720 : 669
Total :  253921
